### 1 - 'absolute_aim_orientation.post_filter_bearing'и 'absolute_aim_orientation.bearing'
### 2 - 'wrap_360(math.deg(current_orient:get_euler_yaw()))'и 'absolute_aim_orientation.bearing'
### 3 - 'wrap_360(math.deg(current_orient:get_euler_yaw()))' и 'absolute_aim_orientation.post_filter_bearing'

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt5
df = pd.read_csv('SCRLOG_00024.csv', sep =';')
df.columns = df.columns.str.strip()

df = df.drop_duplicates().reset_index(drop=True)
df=df.reset_index(drop=True)
df = df.iloc[:-1]

df = df.apply(pd.to_numeric, errors='coerce')#учла наличие пустых значений
df['time_stamp(millis)']= df['time_stamp(millis)']/1000
empty_numbers= df[df['wind_correct.heading'].isnull()].index.tolist()
empty_rows = df.loc[empty_numbers]
empty_numbers.append(len(df))
print(empty_numbers)
print("А всего заходов: ",len(empty_numbers)-1)
###просто с прошлых осталось
df['minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'] = df['wrap_360(math.deg(current_orient:get_euler_yaw()))'] - df['calc_target.heading']
df['minus_absolute_aim_orientation.bearing'] = df['absolute_aim_orientation.bearing'] - df['calc_target.heading']
###

#####мне спокойно, когда все задано изначально. пусть и костыльно)
df['x']= df['calc_target.heading']*0
x_plane = pd.DataFrame(columns=['plane_part'])
y_plane = pd.DataFrame(columns=['plane_part'])
x_point = pd.DataFrame(columns=['point_part'])
y_point = pd.DataFrame(columns=['point_part'])
sinus_part = pd.DataFrame(columns=['sinus_part'])
cosinus_part = pd.DataFrame(columns=['cosinus_part'])
angle = pd.DataFrame(columns=['angle'])
angle_1 = pd.DataFrame(columns=['angle_1'])

x_plane_r = pd.DataFrame(columns=['plane_part'])
y_plane_r = pd.DataFrame(columns=['plane_part'])
x_point_r = pd.DataFrame(columns=['point_part'])
y_point_r = pd.DataFrame(columns=['point_part'])
sinus_part_r = pd.DataFrame(columns=['sinus_part'])
cosinus_part_r = pd.DataFrame(columns=['cosinus_part'])
angle_r = pd.DataFrame(columns=['angle'])
#######


### COPY_1 - поменяла логику. теперь вопрос, какую из разниц хотим посмотреть
print("Какой из заходов на цель хотите посмотреть?")
num = int(input())
print ("Введите критерий: 1 - post_filter/bearing, 2 - yaw/bearing,3 - yaw/post_filter")
kriteriy = input()
df1 = df.iloc[empty_numbers[num-1]+1:empty_numbers[num]]


if (kriteriy == '1'):
    a = 'absolute_aim_orientation.post_filter_bearing'
    b = 'absolute_aim_orientation.bearing'
elif (kriteriy == '2'):
    a = 'wrap_360(math.deg(current_orient:get_euler_yaw()))'
    b = 'absolute_aim_orientation.bearing'
elif (kriteriy == '3'):
    a = 'wrap_360(math.deg(current_orient:get_euler_yaw()))'
    b = 'absolute_aim_orientation.post_filter_bearing'
else:
    print("Чет пошло не так")
    
###Difference
df1['minus_' + a] = df1[a]-df1[b]
df1['r_1'] = np.radians(df1['minus_' + a])

x_plane['plane_part'] = np.sin(df1['r_1'])
y_plane['plane_part'] = np.cos(df1['r_1'])

df1['r_2'] = np.radians(df1[b]*0)
x_point['point_part'] = np.sin(df1['r_2'])
y_point['point_part'] = np.cos(df1['r_2'])


sinus_part['sinus_part'] = x_plane['plane_part']*x_point['point_part'] + y_plane['plane_part']*y_point['point_part']
cosinus_part['cosinus_part'] = x_plane['plane_part']*y_point['point_part'] - x_point['point_part']*y_plane['plane_part']

angle['angle'] = np.arctan2(cosinus_part ['cosinus_part'],sinus_part['sinus_part'])
angle_1['angle_1'] = (angle['angle']**2).expanding().mean()
angle_1['angle_1'] = np.sqrt(angle_1['angle_1'])
df1['difference'] = np.degrees(angle_1['angle_1'])
##########

###Сама визуализация + отображаемые параметры
df1.plot(x="time_stamp(millis)",y=['minus_'+ a,'difference'],linewidth = 3,fontsize=22)
plt.plot(df1["time_stamp(millis)"],df1['x'], color = "black", linestyle ='-',linewidth = 3, alpha = 1, label = b)
plt.legend(bbox_to_anchor=(0.8, 1.1), loc='upper left',fontsize=22)

plt.figtext(0.03, 0.02, df1['difference'].iloc[-1], fontsize=22)
#plt.show()


df1.plot(x="time_stamp(millis)",y=[a,b],linewidth = 3,fontsize=22)
plt.legend(bbox_to_anchor=(0.8, 1.1), loc='upper left',fontsize=22)
plt.figtext(0.03, 0.02, df1['difference'].iloc[-1], fontsize=22)
plt.show()

[0, 1715, 4168, 7329, 13437, 14051, 14830, 15863, 24615]
А всего заходов:  8
Какой из заходов на цель хотите посмотреть?


 1


Введите критерий: 1 - post_filter/bearing, 2 - yaw/bearing,3 - yaw/post_filter


 3


C:\Users\sofia\AppData\Local\Temp\ipykernel_6252\2997857946.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['minus_' + a] = df1[a]-df1[b]
C:\Users\sofia\AppData\Local\Temp\ipykernel_6252\2997857946.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['r_1'] = np.radians(df1['minus_' + a])
C:\Users\sofia\AppData\Local\Temp\ipykernel_6252\2997857946.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in